In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/merged-dataset/merged.sat_Olck
/kaggle/input/merged-dataset/merged.eng_Latn


In [3]:
import pandas as pd
import re

# Load the English and Santali datasets
english_file = '/kaggle/input/merged-dataset/merged.eng_Latn'
santali_file = '/kaggle/input/merged-dataset/merged.sat_Olck'

# Read the datasets
with open(english_file, 'r', encoding='utf-8') as eng_f:
    english_sentences = eng_f.readlines()

with open(santali_file, 'r', encoding='utf-8') as sat_f:
    santali_sentences = sat_f.readlines()

# Create a DataFrame for parallel corpus
df = pd.DataFrame({'english': english_sentences, 'santali': santali_sentences})

# Step 1: Remove rows where 1-to-1 mapping is not there
# First, strip leading/trailing whitespace for better comparison
df['english'] = df['english'].str.strip()
df['santali'] = df['santali'].str.strip()

# Drop rows where English-Santali mappings are not unique (1-to-1 mapping)
df = df.drop_duplicates(subset=['english', 'santali'])

# Step 2: Remove duplicates but keep only one instance (parallel deduplication)
df = df.drop_duplicates()

# Step 3: Trim rows where the length ratio between English and Santali isn't between 0.5 and 1.5
# Define a function to compute the ratio of sentence lengths
def is_in_ratio(row):
    eng_len = len(row['english'].split())
    santali_len = len(row['santali'].split())
    if eng_len == 0 or santali_len == 0:
        return False
    ratio = eng_len / santali_len
    return 0.5 <= ratio <= 1.5

# Apply the length ratio filter
df = df[df.apply(is_in_ratio, axis=1)]

# Step 4: Remove rows where there are English alphabets in the Santali file
def contains_english_characters(text):
    return bool(re.search(r'[a-zA-Z]', text))

# Apply the function to filter rows where Santali contains English characters
df = df[~df['santali'].apply(contains_english_characters)]

# Step 5: Write the cleaned dataset to new files
df['english'].to_csv('cleaned_english.txt', index=False, header=False)
df['santali'].to_csv('cleaned_santali.txt', index=False, header=False)

# Optionally display the cleaned data (for viewing in Kaggle environment)
df.head()


,english,santali
0,The Delhi division of Punjab province formed t...,ᱯᱟᱧᱡᱟᱵᱽ ᱯᱚᱱᱚᱛ ᱨᱮᱭᱟᱜ ᱫᱤᱞᱞᱤ ᱛᱷᱚᱱᱚᱛ ᱦᱚᱨᱤᱭᱟᱱᱟ ᱨᱮᱭᱟ...
1,They teach Pali and Khamti scripts in addition...,ᱩᱱᱠᱩ ᱫᱚ ᱥᱟᱫᱷᱟᱨᱚᱱ ᱥᱮᱪᱮᱫ ᱥᱟᱛᱟᱢ ᱪᱷᱟᱰᱟ ᱦᱚᱸ ᱯᱟᱹᱞᱤ ᱟ...
2,"The western part of the plateau, comprising th...","ᱢᱟᱞᱵᱷᱩᱢᱤ ᱯᱚᱪᱷᱤᱢ ᱯᱟᱦᱟᱴᱟ ᱨᱮ, ᱜᱟᱨᱳ ᱯᱚᱨᱯᱚᱛᱟ ᱴᱚᱴᱷᱟ ..."
3,"Many online sellers, like Amazon and eBay, off...","ᱟᱹᱰᱤᱜᱟᱱ ᱚᱱᱞᱟᱭᱤᱱ ᱟᱠᱷᱨᱤᱧᱤᱭᱟᱹ, ᱡᱮᱞᱮᱠᱟ ᱟᱢᱟᱡᱚᱱ ᱟᱨ ᱤ..."
4,"Notable among them are North East Today, G Plu...","ᱱᱚᱶᱟ ᱢᱩᱫᱽᱨᱮ ᱧᱩᱢᱟᱱᱟᱜ ᱦᱩᱭᱩᱜ ᱠᱟᱱᱟ ᱱᱚᱨᱛᱷ ᱤᱥᱴ ᱴᱩᱰᱮ,..."
